In [3]:
from pytrends.request import TrendReq
import pandas as pd
from tqdm import tqdm, tqdm_gui
import time

/Users/obt/gitRepos/trends/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [19]:

pytrends = TrendReq(hl='ko-KR', tz=1260)

# pytrends.build_payload(kw_list=['pizza', 'bagel'], timeframe=[
#     '2022-09-04 2022-09-10', '2022-09-18 2022-09-24'])
# df = pytrends.multirange_interest_over_time()
# print(df.head())

In [20]:
df2 = pd.read_csv("survey.tsv", sep="\t", encoding='utf-8', header=0)

In [ ]:
df2

,다듬을 말,원어,다듬은 말,제출 점수,의미/용례,sorted11 인덱스
0,"디프 테크, 딥 테크*",deep tech,심층 기술,5.0,"기존의 기술이나 서비스를 개선, 확장하는 것이 아니라 근본적인 혁신을 추구하는 것.",4
1,뷰티 테크,beauty tech,첨단 미용 기술,4.0,첨단 정보 기술을 미용 분야에 적용한 기술.,5
2,슈링크플레이션,shrinkflation←shrink+inflation,"양 줄임, 용량 꼼수",4.0,"줄어들다(shirink)와 인플레이션(inflation)을 합친 말로, 가격은 기존...",6
3,스킴플레이션,skimpflation←skimp+inflation,"질 낮춤, 품질 꼼수",4.0,"인색하다(skimp)와 인플레이션(inflation)을 합친 말로, 값이나 용량은 ...",7
4,번들플레이션,bundleflation←bundle+inflation,묶음 눈속임,4.0,"묶음(bundle)과 인플레이션(inflation)을 합친 말로, 단위 가격으로 비...",8
...,...,...,...,...,...,...
655,"피엘시, PLC",PLC(Power Line Communication(영)),전력선 통신,3.0,전력선을 이용하여 정보를 전송하는 통신 방식<관련 예문>지능형 전력 계량 기반 시설...,825
656,피티,PT(Potential Transformer(영)),전압 변성기,3.0,"고전압을 계량하거나 측정하기 위해 전압을 낮춰 주는 기기<관련 예문>피티, PT(→...",826
657,"피피에이, PPA",PPA(Power Purchase Agreement(영)),전력 구입 계약,1.0,판매 회사에서 전력 공급자로부터 전력을 직접 구입하는 계약<관련 예문>한전에 피피에...,827
658,인터넷 쇼핑몰,Internet Shopping Mall,누리 상가,1.0,가상 공간에 존재하는 가상 상점을 말한다. 인터넷 쇼핑몰은 전통 시장과 같은 시간적...,828


## 검색량 기준 가산점

| 대조군 | 다듬은 | 가산점 |
| ------ | ------ | ------ |
| 100%   | 50%    | -2     |
| 150%   | 100%   | +1     |
| 100%   | 100%   | +2     |
| 80%    | 100%   | +3     |
| 60%    | 100%   | +4     |


In [ ]:
for i in tqdm(range(len(df2))):
  print(f"{df2['다듬을 말'][i]:<50}{df2['다듬은 말'][i]}")

100%|██████████| 660/660 [00:00<00:00, 84910.15it/s]

디프 테크, 딥 테크*                                      심층 기술
뷰티 테크                                             첨단 미용 기술
슈링크플레이션                                           양 줄임, 용량 꼼수
스킴플레이션                                            질 낮춤, 품질 꼼수
번들플레이션                                            묶음 눈속임
밀크플레이션                                            우유발 물가 상승, 유제품 줄인상
퀵 커머스                                             빠른 배달 거래
퀵서비스                                              빠른 배달
마더 팩토리                                            핵심 공장
온 디바이스 에이아이                                       단말형 인공지능
그린 프리미엄                                           친환경 추가비
플러팅                                               호감 표시
패스트 패션                                            빠른 소비 패션
풀필먼트                                              물류 종합 대행
핀플루언서                                             금융 여론 형성자
밸류업                                               가치 향상
스마트 팩토리                                           첨단 자동 공

In [ ]:
# 검색량 비교 및 가산점 계산을 위한 새로운 셀 추가


def calculate_bonus_points(original_ratio, refined_ratio):
  """
  원어와 다듬은 말의 검색 비율에 따라 가산점 계산

  | 대조군 | 다듬은 | 가산점 |
  | ------ | ------ | ------ |
  | 100%   | 50%    | -2     |
  | 150%   | 100%   | +1     |
  | 100%   | 100%   | +2     |
  | 80%    | 100%   | +3     |
  | 60%    | 100%   | +4     |
  """
  if original_ratio == 0 and refined_ratio == 0:
    return 0  # 둘 다 검색량이 없는 경우

  if original_ratio == 0:  # 원어 검색량이 없는 경우
    return 4  # 최대 가산점

  ratio = refined_ratio / original_ratio

  if ratio <= 0.5:  # 다듬은 말이 원어의 50% 이하
    return -2
  elif ratio >= 0.95 and ratio <= 1.05:  # 비슷한 경우 (100%)
    return 2
  elif ratio > 1.05 and refined_ratio >= original_ratio * 1.5:  # 다듬은 말이 훨씬 더 많이 쓰이는 경우
    return 4
  elif ratio > 1.05 and refined_ratio >= original_ratio * 1.25:  # 다듬은 말이 더 많이 쓰이는 경우
    return 3
  elif ratio > 1.05:  # 다듬은 말이 조금 더 쓰이는 경우
    return 2
  elif ratio > 0.67:  # 원어가 조금 더 쓰이는 경우 (150% 수준)
    return 1
  else:
    return 0  # 그 외 케이스


# 검색량과 가산점을 저장할 새 컬럼 추가
df2['원어_검색량'] = 0
df2['다듬은말_검색량'] = 0
df2['검색량_가산점'] = 0

# 진행상황을 표시하기 위한 tqdm

# 검색 데이터를 수집하고 가산점 계산
for i in tqdm(range(len(df2)), desc="검색량 비교 및 가산점 계산"):
  original_term = df2['다듬을 말'][i]
  refined_term = df2['다듬은 말'][i]

  try:
    # 두 단어를 동시에 검색하여 상대적인 검색량 비교
    pytrends.build_payload(
      kw_list=[original_term, refined_term], timeframe='today 1-m', geo='KR')
    result = pytrends.interest_over_time()

    if not result.empty:
      # 평균 검색량 계산
      original_volume = result[original_term].mean()
      refined_volume = result[refined_term].mean()

      # 데이터프레임에 검색량 저장
      df2.at[i, '원어_검색량'] = original_volume
      df2.at[i, '다듬은말_검색량'] = refined_volume

      # 가산점 계산 및 저장
      bonus = calculate_bonus_points(original_volume, refined_volume)
      df2.at[i, '검색량_가산점'] = bonus

  except Exception as e:
    print(f"오류 발생: {original_term} vs {refined_term}: {e}")
    # 오류가 발생하면 다음 단어로 넘어감
    continue

  # API 호출 제한을 위한 짧은 대기 시간
  time.sleep(0.5)

# 결과 확인
print("검색량 및 가산점 계산 완료")

검색량 비교 및 가산점 계산:   0%|          | 0/660 [00:00<?, ?it/s]/Users/obt/gitRepos/trends/.venv/lib/python3.9/site-packages/pytrends/request.py:260: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.fillna(False)
검색량 비교 및 가산점 계산:   1%|          | 5/660 [00:12<25:00,  2.29s/it]/Users/obt/gitRepos/trends/.venv/lib/python3.9/site-packages/pytrends/request.py:260: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.fillna(False)
/var/folders/qh/y7mzyw6s3w300lq_vkfs6gq80000gn/T/ipykernel_36288/1326278687.py:64: FutureWarning: Setting an item of incompatible dt

오류 발생: 에피데믹 vs (감염병) 유행, (감염병) 국가적 유행, (감염병): HTTPSConnectionPool(host='trends.google.com', port=443): Max retries exceeded with url: /trends/api/widgetdata/multiline?req=%7B%22time%22%3A+%222025-02-08+2025-03-08%22%2C+%22resolution%22%3A+%22DAY%22%2C+%22locale%22%3A+%22ko%22%2C+%22comparisonItem%22%3A+%5B%7B%22geo%22%3A+%7B%22country%22%3A+%22KR%22%7D%2C+%22complexKeywordsRestriction%22%3A+%7B%22keyword%22%3A+%5B%7B%22type%22%3A+%22BROAD%22%2C+%22value%22%3A+%22%5Cuc5d0%5Cud53c%5Cub370%5Cubbf9%22%7D%5D%7D%7D%2C+%7B%22geo%22%3A+%7B%22country%22%3A+%22KR%22%7D%2C+%22complexKeywordsRestriction%22%3A+%7B%22keyword%22%3A+%5B%7B%22type%22%3A+%22BROAD%22%2C+%22value%22%3A+%22%5Cuac10%5Cuc5fc%5Cubcd1++%5Cuc720%5Cud589%2C++%5Cuac10%5Cuc5fc%5Cubcd1++%5Cuad6d%5Cuac00%5Cuc801+%5Cuc720%5Cud589%2C++%5Cuac10%5Cuc5fc%5Cubcd1%22%7D%5D%7D%7D%5D%2C+%22requestOptions%22%3A+%7B%22property%22%3A+%22%22%2C+%22backend%22%3A+%22IZG%22%2C+%22category%22%3A+0%7D%2C+%22userConfig%22%3A+%7B%22userType%22%3A+%22US

검색량 비교 및 가산점 계산:  43%|████▎     | 282/660 [10:00<19:55,  3.16s/it]

오류 발생: 팬데믹 vs (감염병) 세계적 유행: HTTPSConnectionPool(host='trends.google.com', port=443): Read timed out. (read timeout=2)


/Users/obt/gitRepos/trends/.venv/lib/python3.9/site-packages/pytrends/request.py:260: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.fillna(False)
검색량 비교 및 가산점 계산:  43%|████▎     | 283/660 [10:06<24:19,  3.87s/it]/Users/obt/gitRepos/trends/.venv/lib/python3.9/site-packages/pytrends/request.py:260: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.fillna(False)
검색량 비교 및 가산점 계산:  43%|████▎     | 285/660 [10:13<22:48,  3.65s/it]/Users/obt/gitRepos/trends/.venv/lib/python3.9/site-packages/pytrends/request.py:260: FutureWarning: Downcasting object 

...

오류 발생: 케어 팜 vs 치유 농장: HTTPSConnectionPool(host='trends.google.com', port=443): Read timed out. (read timeout=2)


검색량 비교 및 가산점 계산:  51%|█████     | 337/660 [12:24<16:40,  3.10s/it]

오류 발생: 라키비움 vs 복합 문화 공간: HTTPSConnectionPool(host='trends.google.com', port=443): Max retries exceeded with url: /trends/api/explore?hl=ko-KR&tz=1260&req=%7B%22comparisonItem%22%3A+%5B%7B%22keyword%22%3A+%22%5Cub77c%5Cud0a4%5Cube44%5Cuc6c0%22%2C+%22time%22%3A+%22today+1-m%22%2C+%22geo%22%3A+%22KR%22%7D%2C+%7B%22keyword%22%3A+%22%5Cubcf5%5Cud569+%5Cubb38%5Cud654+%5Cuacf5%5Cuac04%22%2C+%22time%22%3A+%22today+1-m%22%2C+%22geo%22%3A+%22KR%22%7D%5D%2C+%22category%22%3A+0%2C+%22property%22%3A+%22%22%7D (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x1178a96a0>, 'Connection to trends.google.com timed out. (connect timeout=2)'))


/Users/obt/gitRepos/trends/.venv/lib/python3.9/site-packages/pytrends/request.py:260: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.fillna(False)
검색량 비교 및 가산점 계산:  52%|█████▏    | 342/660 [12:38<13:55,  2.63s/it]/Users/obt/gitRepos/trends/.venv/lib/python3.9/site-packages/pytrends/request.py:260: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.fillna(False)
검색량 비교 및 가산점 계산:  52%|█████▏    | 343/660 [12:41<13:24,  2.54s/it]/Users/obt/gitRepos/trends/.venv/lib/python3.9/site-packages/pytrends/request.py:260: FutureWarning: Downcasting object 

검색량 및 가산점 계산 완료


In [23]:
df2

,다듬을 말,원어,다듬은 말,제출 점수,의미/용례,sorted11 인덱스,원어_검색량,다듬은말_검색량,검색량_가산점
0,"디프 테크, 딥 테크*",deep tech,심층 기술,5.0,"기존의 기술이나 서비스를 개선, 확장하는 것이 아니라 근본적인 혁신을 추구하는 것.",4,0.000000,6.0,4
1,뷰티 테크,beauty tech,첨단 미용 기술,4.0,첨단 정보 기술을 미용 분야에 적용한 기술.,5,0.000000,0.0,0
2,슈링크플레이션,shrinkflation←shrink+inflation,"양 줄임, 용량 꼼수",4.0,"줄어들다(shirink)와 인플레이션(inflation)을 합친 말로, 가격은 기존...",6,0.000000,0.0,0
3,스킴플레이션,skimpflation←skimp+inflation,"질 낮춤, 품질 꼼수",4.0,"인색하다(skimp)와 인플레이션(inflation)을 합친 말로, 값이나 용량은 ...",7,0.000000,0.0,0
4,번들플레이션,bundleflation←bundle+inflation,묶음 눈속임,4.0,"묶음(bundle)과 인플레이션(inflation)을 합친 말로, 단위 가격으로 비...",8,0.000000,0.0,0
...,...,...,...,...,...,...,...,...,...
655,"피엘시, PLC",PLC(Power Line Communication(영)),전력선 통신,3.0,전력선을 이용하여 정보를 전송하는 통신 방식<관련 예문>지능형 전력 계량 기반 시설...,825,0.000000,0.0,0
656,피티,PT(Potential Transformer(영)),전압 변성기,3.0,"고전압을 계량하거나 측정하기 위해 전압을 낮춰 주는 기기<관련 예문>피티, PT(→...",826,67.310345,0.0,-2
657,"피피에이, PPA",PPA(Power Purchase Agreement(영)),전력 구입 계약,1.0,판매 회사에서 전력 공급자로부터 전력을 직접 구입하는 계약<관련 예문>한전에 피피에...,827,0.000000,0.0,0
658,인터넷 쇼핑몰,Internet Shopping Mall,누리 상가,1.0,가상 공간에 존재하는 가상 상점을 말한다. 인터넷 쇼핑몰은 전통 시장과 같은 시간적...,828,23.068966,0.0,-2


In [24]:
df2.to_csv('searched.csv', sep=",", na_rep="NaN")

In [4]:
df2 = pd.read_csv("searched.csv", sep=",", encoding='utf-8', header=0)

In [5]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [6]:
df2.sort_values(by='다듬은말_검색량', ascending=False)[
    ['다듬을 말', '다듬은 말', '원어_검색량', '다듬은말_검색량', '검색량_가산점']]

,다듬을 말,다듬은 말,원어_검색량,다듬은말_검색량,검색량_가산점
393,로깅,기록,1.068966,90.310345,4
494,코칭,지도,0.965517,81.034483,4
567,시즈닝,양념,6.068966,76.896552,4
394,벤트,배출,15.000000,75.310345,4
496,트리포트,화분,0.000000,69.103448,4
511,루프톱,옥상,0.000000,67.620690,4
478,이북,전자책,57.034483,58.068966,2
152,언택트,비대면,4.275862,56.344828,4
635,레짐,체제,1.724138,48.344828,4
470,뷰,전망,86.413793,47.586207,0
